In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from scipy.interpolate import interpolate, griddata
from mpl_toolkits.axes_grid.axislines import Subplot
from skimage.transform import radon, iradon, iradon_sart
from skimage.transform import rescale
from skimage import filters

with open('ionos.dat', "r") as ins:
    lines = []
    for line in ins:
        lines.append(line)
        
y_string = ""
for line in (range(2,19)):
    y_string = y_string + lines[line]
    alt = y_string.split()

x_string = ""
for line in (range(21,38)):
    x_string = x_string + lines[line]
    den = x_string.split()

#1D Plot
#plt.plot(den, alt)
#plt.ylabel("Altitude (km)")
#plt.xlabel("Electron Densities (cm^-3)")
#plt.title("Electron Density as Function of Altitude")
#plt.show()

In [3]:
N = 500 # number of points moving along limb
data_y = np.array([float(a) for a in alt])# altitudes
data_x = np.array([float(d) for d in den])# electron densities
max_Rx = data_x.max()
min_Rx = data_x.min()
max_Ry = data_y.max()
min_Ry = data_y.min()

xs = np.linspace(min_Rx, max_Rx, N)
ys = np.linspace(min_Ry, max_Ry, N)
x,y = np.meshgrid(xs,ys)

#r = np.sqrt(x**2 + y**2)

f = lambda x: griddata(data_y,data_x,x, method='linear')

#plt.imshow(f(r), cmap=plt.cm.viridis)
#plt.show()
#f(r).shape

In [4]:
#Now time to do the limited angle - guess of about 160-175 with two varying plumes
#image = f(r)
N_lat = 500 #latitudes
plume = [f(ys)]*N_lat

#adjust plume widths:
n=1
m=8

plume1= n
for i in range(int(N_lat/3 - plume1/2), int(N_lat/3 + plume1/2)):
    plume[i]=[0]*len(ys)
    
plume2 = m
for j in range(int(N_lat/6 - plume2/2), int(N_lat/6 + plume2/2)):
    plume[j]=[0]*len(ys)

image = np.hstack(plume)
image = np.reshape(image, (N_lat, len(ys)))
image = image.T

print(round(N_lat/6), 'points along the latitude')
print(round(N_lat/2), 'points along the latitude')

#original plume width
#total width along y-axis = 1000 km/500 px in column

km_px = 1000/500 #1000km/500px*plume width px

p1 = km_px*plume1
p2 = km_px*plume2


print('plume widths =',p1,',', p2,'km')


83 points along the latitude
250 points along the latitude
plume widths = 2.0 , 16.0 km


In [5]:
#normalize colormap
my_norm = Normalize(image.min(), image.max())
#Original image plot

plt.title("Electron Density")
plt.xlabel("Latitude")
plt.ylabel('Altitude (km)')
plt.imshow(image, cmap=plt.cm.viridis, aspect = 'auto', norm = my_norm)
plt.gca().invert_yaxis()
ticks = np.arange(0, 500, 49)
labels = [int(round((ys[i])/100)*100) for i in ticks]
plt.yticks(ticks, labels)
#plt.show()

([<matplotlib.axis.YTick at 0x2c4ad5e0e10>,
 <a list of 11 Text yticklabel objects>)

In [19]:
#Zenith Angle from altitudes 

Re = 6371 #km
z = list(range(120,360,1)) #km
h = 375 #km observation height
p = np.asarray([Re])+z 
q = Re+h #km
t = np.divide(p,q)
tanpts = (np.arccos(t)+np.pi/2)
limb = 180 - np.degrees(tanpts)

#print(limb)
#limb angles are 73-83. Cool.


In [20]:
#FULL ANGLE 
theta_full = np.arange(180)
sinogram = radon(image, theta_full)

In [21]:
#full-angle singoram
plt.title("Full Angle Sinogram")
plt.ylabel("Projection Position Along Track (px)")
plt.xlabel("Projection Angle (deg)") #how far from center of image perpendicular to line you're looking at 
plt.imshow(sinogram, cmap=plt.cm.viridis, aspect = 'auto')
my_xticks = np.arange(0,181,20)
plt.xticks(my_xticks)
plt.gca().invert_yaxis()
plt.show()


In [22]:
#full backprojection 

ir=iradon(sinogram, theta_full, circle= False)
plt.title("Full Angle Backprojection")
plt.xlabel("Latitude")
plt.ylabel("Altitude (km)")
plt.imshow(ir, cmap=plt.cm.viridis, aspect = 'auto', norm=my_norm)
plt.gca().invert_yaxis()
x_ticks = np.arange(0,501,50)
ticks = np.arange(0, 500, 49)
labels = [int(round((ys[i])/100)*100) for i in ticks]
plt.yticks(ticks, labels)
plt.xticks(x_ticks)
plt.show()

In [23]:
#LITES ONLY ANGLES
nadir = []
theta_lites = np.hstack([nadir, limb])
sinogram_lites = radon(image,theta_lites)


In [27]:
#LITES ONLY SINOGRAM

plt.title("Limb Sinogram")
plt.ylabel("Projection Position Along Track (px)")
plt.xlabel("Projection Angle (deg)") #how far from center of image perpendicular to line you're looking at 
print(sinogram_lites.shape)
sinogram_disp = sinogram_lites.copy()
#for t in range(0,72):
    #sinogram_disp[:,t] = [0]*708
#for t in range(84,240):
    #sinogram_disp[:,t] = [0]*708
plt.imshow(sinogram_disp, cmap=plt.cm.viridis, aspect = 'auto')
my_xticks = np.arange(0,240,40)
labels = [str(round(theta_lites[i],0)) for i in range(0,240,40)]
plt.xticks(my_xticks, labels)
plt.gca().invert_yaxis()
plt.show()


(708, 240)


In [28]:
#LITES ONLY BP
ir=iradon(sinogram_lites, theta_lites, circle= False)
plt.title("Limb Backprojection")
plt.xlabel("Latitude")
plt.ylabel("Altitude, (km)")
plt.imshow(ir, cmap=plt.cm.viridis, aspect = 'auto')
plt.gca().invert_yaxis()
labels = [int(round((ys[i])/100)*100) for i in ticks]
x_ticks = np.arange(0, 501, 50)
plt.yticks(x_ticks, labels)
plt.show()

In [26]:
#Lites and Tip
nadir = np.arange(0,6,1)
theta_both = np.hstack([nadir, limb])
sinogram_both = radon(image, theta_both)

In [39]:
#Lites and Tip

plt.title("Limb and Nadir Sinogram")
plt.ylabel("Projection Position Along Track (px)")
plt.xlabel("Projection Angle (deg)") #how far from center of image perpendicular to line you're looking at 


sinogram_disp = sinogram.copy()
for t in range(6,72):
    sinogram_disp[:,t] = [0]*708
for t in range(84,180):
    sinogram_disp[:,t] = [0]*708
plt.imshow(sinogram_disp, cmap=plt.cm.viridis, aspect = 'auto')
plt.gca().invert_yaxis()
my_xticks = np.arange(0,180,40)
labels = [str(i) for i in range(0,180,40)]
plt.xticks(my_xticks, labels)
plt.show()




In [29]:
#LITES AND TIP BP
ir=iradon(sinogram_both, theta_both, circle= False)
plt.title("Limb/Nadir Backprojection")
plt.xlabel("Latitude")
plt.ylabel("Altitude, (km)")
plt.imshow(ir, cmap=plt.cm.viridis, aspect = 'auto')
plt.gca().invert_yaxis()
ticks = np.arange(0, 500, 49)
labels = [int(round((ys[i])/100)*100) for i in ticks]
plt.yticks(ticks, labels)
plt.show()


In [51]:
#edge plot

from skimage import feature

#Apply Gaussian blur technique coupled with low/high thresholding

edges = feature.canny(ir, sigma = 1.0, low_threshold = 0.52, high_threshold = 0.996, use_quantiles = True)
ticks = np.arange(0, 500, 49)
x_ticks = np.arange(0,501,50)
labels = [int(round((ys[i])/100)*100) for i in ticks]
plt.title('Plume Edge Plot, Sigma 1 with Low/High Thresholding')
plt.xlabel('Latitude')
plt.ylabel('Altitude (km)')
plt.imshow(edges, cmap=plt.cm.gray, aspect = 'auto')
plt.gca().invert_yaxis()
plt.xticks(x_ticks)
plt.yticks(ticks, labels)
plt.show()

In [45]:
#2-8 km plume width

row = 200
edgex =[]
for x in range(0,200):
    if edges[row,x]:
        edgex.append(x)
for x in range(201,400):
    if edges[row,x]:
        edgex.append(x)
for i in range(0,len(edgex)-1):
    d = int(edgex[i+1] - edgex[i])
    c = 1000/500 #2km/px
    w= c*d
    if 0 < w < 20:
        print('Measured width from edge plot', w, 'km')

rowkm = row*1000/500

print('Altitude =',rowkm, 'km')



Measured width from edge plot 16.0 km
Measured width from edge plot 4.0 km
Altitude = 400.0 km


In [46]:
#Trapezoid rule; estimate for thickness of the band
#chapman function to find width of band
#height of the band - estimate scale height rho= N_max*e^(-1/2(1-(z-z_max/H)-e^-(z-z_max/4))); H = scale height/peak value

#estimate of scale height from density peak. How tall it would be if density was constant
import scipy.integrate as sc
x = data_x
y = data_y
I= sc.trapz(x[::-1],y[::-1])
N = x.max()

I/N

a = round((1000/500)*(I/N))

print('Scale height', a , 'km')




Scale height 566.0 km


In [47]:
#Width of band by function np.argmax
#plume = [f(ys)]*N_lat

i= np.asarray(plume)[:,0].argmax()
z_max = z[i]

width_of_band = 1000/500*z_max

print('Width of band', width_of_band, 'km')

Width of band 240.0 km


In [169]:
#res index

#number of counts/sec
n = 1000
#delta time (choose)
d_t = 100 #secs
#intensity at 1356 Angstroms
I = 10
#change in angle 10 degrees in rads 
d_th = np.pi/18


Resolution = d_th*n/(d_t*I*d_t) #counts/per sec *rayleighs

print(Resolution)


0.0017453292519943296


In [45]:
#magntiude of brightness/intensity for 1356; resolution proportional intensity*number of counts per sec 
#ask susanna for #counts/sec
#angles chosen effects resolution I*N*deltaT; deltaT is elapsed time when counting - you can choose that. resolution will change over time 

#delta(theta)= 10 deg


#github code 
